## Exercício LangChain e Gemini - ALURA

- Exercício proposto pro curso de langChain e python - ALURA - Cração dde cadeia simples: 
- Emoções (Roda das Emoções de Plutchik)
- Distorções cognitivas
- Cartões de enfrentamento

- No exercício envolvi assuntos de interesse pessoal e tive por objetivo usar o conceito de Simple Sequencial Chain para analisar o conteúdo emocional e distorções cognitivas de um trecho da crônica "Se eu fosse eu" de Clarice Lispector e a partir da análise propor cartões de enfrentamento para pensamentos disfuncionais.   

### Imports

In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

### Carregamento da Chave API -> Utilização do Gemini 

In [2]:
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

### Instancia da LLM

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    google_api_key=api_key
)

### Definição dos templates de prompts

In [4]:
modelo_emocoes = ChatPromptTemplate.from_template(
    "Identifique as emoções presentes na seguinte texto conforme a teoria da roda das emoções. texto: {texto}"
)

modelo_distorcoes_cognitivas = ChatPromptTemplate.from_template(
    "Identifique as distorções cognitivas a seguinte analise emocional: {emoções}"
)

modelo_pensamentos_automaticos = ChatPromptTemplate.from_template(
    "Sugira pensamentos saudáveis de enfrentamento dadas as seguintes distorções cognitivas: {distorções}. Vc deve montar um esquema tipo cartões de enfrentamento, estilo distorção -> pensamento saudável"
)


### Criação das Cadeias

In [5]:
cadeia_emocoes = LLMChain(
    llm=llm,
    prompt=modelo_emocoes
)
cadeia_distorcoes = LLMChain(
    llm=llm,
    prompt=modelo_distorcoes_cognitivas
)

cadeia_pensamentos = LLMChain(
    llm=llm,
    prompt=modelo_pensamentos_automaticos
)

# criação de uma sequência de cadeias -  simple sequential chain
# usa a saída de uma cadeia como entrada da próxima
# a saída da cadeia_emocoes é a entrada da cadeia_distorcoes, e assim por diante

cadeia = SimpleSequentialChain(
    chains=[cadeia_emocoes, cadeia_distorcoes, cadeia_pensamentos],
    verbose=True # pra ter detalhes da execução passo a passo no terminal
)


C:\Users\syann\AppData\Local\Temp\ipykernel_16980\1255668256.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  cadeia_emocoes = LLMChain(


### Execução da Cadeia 

In [6]:
texto = """E não me sinto bem. Experimente: se você fosse você, como seria e o que faria? 
Logo de início se sente um constrangimento: a mentira em que nos acomodamos acabou de ser 
levemente locomovida do lugar onde se acomodara. No entanto já li biografias de pessoas que 
de repente passavam a ser elas mesmas, e mudavam inteiramente de vida. Acho que se eu fosse 
realmente eu, os amigos não me cumprimentariam na rua porque até minha fisionomia teria mudado. Como? Não sei."""

resultado = cadeia.invoke(texto)
print(resultado)




> Entering new SimpleSequentialChain chain...
O texto expressa principalmente **constrangimento** e **medo**, com nuances de **tristeza** e possivelmente **insegurança**. Vamos analisar pela Roda das Emoções de Plutchik:

* **Constrangimento:**  A frase "E não me sinto bem" e a descrição da mentira levemente deslocada apontam diretamente para uma sensação de desconforto e vergonha, características do constrangimento.  A mentira, provavelmente sobre a própria identidade ou comportamento, gera um sentimento de exposição e vulnerabilidade.

* **Medo:** A ideia de que "os amigos não me cumprimentariam na rua porque até minha fisionomia teria mudado" revela um medo profundo de rejeição e isolamento social caso a pessoa se mostrasse como realmente é.  A incerteza ("Como? Não sei.") sobre como seria essa mudança física reforça esse medo do desconhecido.

* **Tristeza:** A frase inicial "E não me sinto bem" sugere um estado de mal-estar geral que pode ser interpretado como tristeza, uma sens

### Utilização de Parser

Proposta:
- Analisar emoções e retornar uma estrutura com a lista de emoções e a justificativa das emoções
- analisar sentimentos relacionados com as emoções detectadas anteriormente


In [7]:
from langchain_core.pydantic_v1 import Field, BaseModel
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

## criação da classe pro parser
class AnaliseEmocional(BaseModel):
    emoções: list[str] = Field(description="Emoções identificadas no texto")
    justificativa: str = Field(description="Justificativa para as emoções identificadas")

## definição do parser
parser = JsonOutputParser(pydantic_object=AnaliseEmocional)

## modelos prompts com o parser
modelo_emocoes = PromptTemplate(
    template ="""Identifique as emoções presentes na seguinte texto conforme a teoria da roda das emoções.
    Texto: {texto}
    {formatacao_de_saida}
    """,
    input_variables=["texto"],
    partial_variables={"formatacao_de_saida": parser.get_format_instructions()}
) 


classificacao_sentimentos = ChatPromptTemplate.from_template(
    "Classifique a potcentagem dos sentimentos(positivo, negativo ou neutro) presentas na lista de emoções. Ex: 'Na lista estão presentes 50% de emoções negativas e 50% de positivas.' Emoções: {emoções}"
)

## criação das cadeia, uma a uma 

cadeia_emocoes = LLMChain(
    llm=llm,
    prompt=modelo_emocoes
)

cadeia_classificacao_sentimentos = LLMChain(
    llm=llm,
    prompt=classificacao_sentimentos
)

cadeia = SimpleSequentialChain(
    chains= [cadeia_emocoes, cadeia_classificacao_sentimentos],
    verbose=True
)


# criação de uma sequência de cadeias -  simple sequential chain
# usa a saída de uma cadeia como entrada da próxima
# a saída da cadeia_emocoes é a entrada da cadeia_distorcoes, e assim por diante

cadeia = SimpleSequentialChain(
    chains= [cadeia_emocoes, cadeia_classificacao_sentimentos],
    verbose=True
)

# execução da cadeia
texto = """E não me sinto bem. Experimente: se você fosse você, como seria e o que faria? 
Logo de início se sente um constrangimento: a mentira em que nos acomodamos acabou de ser 
levemente locomovida do lugar onde se acomodara. No entanto já li biografias de pessoas que 
de repente passavam a ser elas mesmas, e mudavam inteiramente de vida. Acho que se eu fosse 
realmente eu, os amigos não me cumprimentariam na rua porque até minha fisionomia teria mudado. Como? Não sei."""

resultado = cadeia.invoke(texto)

c:\Users\syann\Downloads\Códigos\Estudos_langChain_Alura\langchain\Lib\site-packages\IPython\core\interactiveshell.py:3672: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)




> Entering new SimpleSequentialChain chain...
```json
{
  "emoções": ["constrangimento", "medo", "insegurança", "desconforto"],
  "justificativa": "O texto expressa um sentimento de mal-estar inicial (\"E não me sinto bem\"), que pode ser classificado como desconforto. A frase \"a mentira em que nos acomodamos acabou de ser levemente locomovida do lugar onde se acomodara\" indica constrangimento pela revelação ou possibilidade de revelação de uma verdade escondida. A insegurança e o medo são evidentes na imaginação de uma mudança radical na vida (\"se eu fosse realmente eu, os amigos não me cumprimentariam na rua porque até minha fisionomia teria mudado\"),  sugerindo o receio de rejeição e da consequência de ser autêntico. A incerteza sobre como essa mudança se manifestaria (\"Como? Não sei\") reforça a sensação de desconforto e insegurança."
}
```
Na lista estão presentes 100% de emoções negativas.  Todas as emoções listadas (constrangimento, medo, insegurança, desconforto) são con